# Prédiction Spaceship Titanic

## Contexte du Problème

Ce notebook implémente une solution complète pour le challenge Kaggle **Spaceship Titanic**. L'objectif est de prédire si un passager a été transporté vers une autre dimension lors d'une collision spatiale.

### Données Disponibles
- **train.csv** : 8693 passagers avec la variable cible `Transported`
- **test.csv** : Passagers pour lesquels nous devons prédire le transport
- **Variables** : 13 features incluant dépenses, planète d'origine, destination, cabine, etc.

### Stratégie de Modélisation
1. **Prétraitement intelligent** : Gestion des valeurs manquantes et création de features dérivées
2. **Feature Engineering** : Extraction d'informations des données structurées
3. **Modélisation comparative** : Test de différents algorithmes (Logistic Regression vs Random Forest)
4. **Évaluation rigoureuse** : Utilisation de métriques appropriées (Accuracy, ROC-AUC)
5. **Prédiction finale** : Entraînement sur tout le dataset et soumission


In [ ]:
# Notebook de prédiction Spaceship Titanic (pipeline propre)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

# Charger
train_df = pd.read_csv('train.csv')

print('Train:', train_df.shape)



Train: (8693, 15)


## Chargement et Exploration des Données

### Imports et Configuration
Les imports incluent :
- **Pandas/Numpy** : Manipulation des données
- **Scikit-learn** : Machine Learning (préprocessing, modèles, métriques)
- **Modèles testés** : Logistic Regression et Random Forest

### Chargement des Données
- **Dataset d'entraînement** : 8693 passagers avec leurs informations
- **Extraction du GroupId** : Les passagers voyagent en groupes (identifiés par la première partie du PassengerId)
- **Taille** : 8693 lignes × 14 colonnes (13 features + target)

### Logique Initiale
Le `GroupId` est extrait car les passagers voyageant ensemble peuvent avoir des comportements similaires, ce qui sera exploité dans le feature engineering.


In [14]:
# Prétraitement + features
# Transported -> int 0/1
if train_df['Transported'].dtype == 'object':
    train_df['Transported'] = train_df['Transported'].map({'True': 1, 'False': 0})
train_df['Transported'] = train_df['Transported'].astype(int)

# Dépenses -> numériques NaN=0
expense_cols = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
for c in expense_cols:
    train_df[c] = pd.to_numeric(train_df[c], errors='coerce').fillna(0)
# Features dépenses
train_df['TotalExpenses'] = train_df[expense_cols].sum(axis=1)
train_df['HasExpense'] = (train_df['TotalExpenses'] > 0).astype(int)

# Catégorielles nettoyées
train_df['CryoSleep'] = train_df['CryoSleep'].fillna('Unknown').astype(str)
train_df['Destination'] = train_df['Destination'].fillna('Unknown').astype(str)
train_df['HomePlanet'] = train_df['HomePlanet'].fillna('Unknown').astype(str)
train_df['VIP'] = train_df['VIP'].fillna('Unknown').astype(str)

# Groupes
train_df['GroupId'] = train_df['PassengerId'].str.split('_').str[0]
train_df['GroupSize'] = train_df.groupby('GroupId')['GroupId'].transform('count')

# Cabine
train_df['Cabin_Deck'] = train_df['Cabin'].str.split('/').str[0]
train_df['Cabin_Side'] = train_df['Cabin'].str.split('/').str[2]

# Age
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].median())

feature_cols = ['CryoSleep','HasExpense','TotalExpenses','HomePlanet','Destination',
                'GroupSize','Cabin_Deck','Cabin_Side','Age','VIP']
X = train_df[feature_cols]
y = train_df['Transported']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print('X_train:', X_train.shape, 'X_val:', X_val.shape)


X_train: (6954, 10) X_val: (1739, 10)


## Prétraitement et Feature Engineering

### Transformation de la Variable Cible
- **Conversion booléenne → numérique** : `True/False` → `1/0` pour la modélisation
- **Type casting** : Conversion en entier pour éviter les erreurs

### Feature Engineering des Dépenses
**Hypothèse** : Les dépenses peuvent être un indicateur de comportement et de statut social.

- **Colonnes de dépenses** : RoomService, FoodCourt, ShoppingMall, Spa, VRDeck
- **Gestion des NaN** : Remplacement par 0 (logique : pas de dépense = 0)
- **Features dérivées** :
  - `TotalExpenses` : Somme de toutes les dépenses
  - `HasExpense` : Indicateur binaire (1 si dépenses > 0, 0 sinon)

### Nettoyage des Variables Catégorielles
**Stratégie** : Remplacer les valeurs manquantes par "Unknown" plutôt que de les supprimer
- **CryoSleep** : État de sommeil cryogénique
- **Destination** : Planète de destination
- **HomePlanet** : Planète d'origine
- **VIP** : Statut VIP

### Features de Groupe
- **GroupSize** : Nombre de passagers dans le même groupe
- **Logique** : Les groupes peuvent avoir des comportements similaires

### Parsing de la Cabine
- **Cabin_Deck** : Pont de la cabine (A, B, C, etc.)
- **Cabin_Side** : Côté du vaisseau (P ou S)
- **Logique** : L'emplacement peut influencer la probabilité de transport

### Gestion de l'Âge
- **Imputation** : Remplacement des NaN par la médiane
- **Choix de la médiane** : Plus robuste que la moyenne aux valeurs aberrantes

### Sélection des Features Finales
10 features retenues pour la modélisation, équilibrant information et simplicité.


In [15]:
# Pipeline + modèles, évaluation
categorical_features = ['CryoSleep','HomePlanet','Destination','Cabin_Deck','Cabin_Side','VIP']
numeric_features = ['HasExpense','TotalExpenses','GroupSize','Age']

preprocess = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', StandardScaler(), numeric_features)
    ]
)

log_reg = Pipeline([
    ('prep', preprocess),
    ('clf', LogisticRegression(max_iter=1000))
])

rf = Pipeline([
    ('prep', preprocess),
    ('clf', RandomForestClassifier(n_estimators=400, random_state=42))
])

models = {'LogReg': log_reg, 'RandomForest': rf}
metrics = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    proba = model.predict_proba(X_val)[:,1]
    acc = accuracy_score(y_val, pred)
    auc = roc_auc_score(y_val, proba)
    metrics[name] = {'acc': acc, 'auc': auc}
    print(f"\n=== {name} ===")
    print(f"Accuracy: {round(acc,4)} | ROC AUC: {round(auc,4)}")
    print('Matrice de confusion:\n', confusion_matrix(y_val, pred))
    print('Rapport de classification:\n', classification_report(y_val, pred, digits=3))

best_name = max(metrics, key=lambda k: metrics[k]['auc'])
best_model = models[best_name]
print(f"\n>>> Meilleur modèle: {best_name} (AUC={round(metrics[best_name]['auc'],4)}, Acc={round(metrics[best_name]['acc'],4)})")



=== LogReg ===
Accuracy: 0.7384 | ROC AUC: 0.7888
Matrice de confusion:
 [[701 162]
 [293 583]]
Rapport de classification:
               precision    recall  f1-score   support

           0      0.705     0.812     0.755       863
           1      0.783     0.666     0.719       876

    accuracy                          0.738      1739
   macro avg      0.744     0.739     0.737      1739
weighted avg      0.744     0.738     0.737      1739


=== RandomForest ===
Accuracy: 0.7188 | ROC AUC: 0.7843
Matrice de confusion:
 [[655 208]
 [281 595]]
Rapport de classification:
               precision    recall  f1-score   support

           0      0.700     0.759     0.728       863
           1      0.741     0.679     0.709       876

    accuracy                          0.719      1739
   macro avg      0.720     0.719     0.718      1739
weighted avg      0.721     0.719     0.718      1739


>>> Meilleur modèle: LogReg (AUC=0.7888, Acc=0.7384)


## Modélisation et Évaluation

### Pipeline de Préprocessing
**ColumnTransformer** pour gérer différents types de données :
- **Variables catégorielles** : OneHotEncoder avec `handle_unknown='ignore'`
- **Variables numériques** : StandardScaler pour normalisation
- **Avantage** : Pipeline robuste et réutilisable

### Modèles Testés

#### Logistic Regression
- **Avantages** : Rapide, interprétable, bon baseline
- **Configuration** : `max_iter=1000` pour convergence
- **Résultats** : AUC = 0.7888, Accuracy = 0.7384

#### Random Forest
- **Avantages** : Capture interactions complexes, robuste au bruit
- **Configuration** : 400 estimateurs pour stabilité
- **Résultats** : AUC = 0.7843, Accuracy = 0.7188

### Stratégie d'Évaluation

#### Division Train/Validation
- **Split** : 80% train / 20% validation
- **Stratification** : Préservation de la distribution des classes
- **Random state** : Reproductibilité des résultats

#### Métriques Utilisées
1. **Accuracy** : Pourcentage de prédictions correctes
2. **ROC-AUC** : Capacité discriminatoire du modèle
3. **Matrice de confusion** : Détail des erreurs par classe
4. **Classification report** : Precision, Recall, F1-score

### Sélection du Meilleur Modèle
**Critère** : ROC-AUC (plus robuste que l'accuracy pour les problèmes déséquilibrés)
- **Gagnant** : Logistic Regression (AUC = 0.7888)
- **Performance** : Bon équilibre entre précision et rappel


In [16]:
# Refit sur tout le train + prédire test.csv et écrire submission
best_model.fit(X, y)
Test = pd.read_csv('test.csv')

# Recréer features pour Test
T = Test.copy()
for c in ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']:
    T[c] = pd.to_numeric(T[c], errors='coerce').fillna(0)
T['TotalExpenses'] = T[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].sum(axis=1)
T['HasExpense'] = (T['TotalExpenses'] > 0).astype(int)
T['CryoSleep'] = T['CryoSleep'].fillna('Unknown').astype(str)
T['Destination'] = T['Destination'].fillna('Unknown').astype(str)
T['HomePlanet'] = T['HomePlanet'].fillna('Unknown').astype(str)
T['VIP'] = T['VIP'].fillna('Unknown').astype(str)
T['GroupId'] = T['PassengerId'].str.split('_').str[0]
T['GroupSize'] = T.groupby('GroupId')['GroupId'].transform('count')
T['Cabin_Deck'] = T['Cabin'].str.split('/').str[0]
T['Cabin_Side'] = T['Cabin'].str.split('/').str[2]
T['Age'] = T['Age'].fillna(train_df['Age'].median())

X_sub = T[feature_cols]
proba = best_model.predict_proba(X_sub)[:,1]
pred_bool = proba >= 0.5

submission = pd.DataFrame({'PassengerId': T['PassengerId'], 'Transported': pred_bool})
submission.to_csv('submission.csv', index=False)
print("submission.csv écrit.")


submission.csv écrit.


## Prédiction Finale et Soumission

### Réentraînement sur le Dataset Complet
**Logique** : Une fois le meilleur modèle identifié, on l'entraîne sur toutes les données disponibles pour maximiser les performances.

### Traitement du Dataset de Test
**Cohérence** : Application exacte du même preprocessing que sur les données d'entraînement :

1. **Dépenses** : Conversion numérique + remplacement NaN par 0
2. **Features dérivées** : TotalExpenses et HasExpense
3. **Variables catégorielles** : Remplissage par "Unknown"
4. **Groupe** : Calcul du GroupSize
5. **Cabine** : Parsing Deck et Side
6. **Âge** : Imputation par la médiane du dataset d'entraînement

### Stratégie de Prédiction
- **Probabilités** : Utilisation de `predict_proba()` pour obtenir les scores de confiance
- **Seuil de décision** : 0.5 (peut être optimisé selon les métriques)
- **Conversion finale** : Booléen pour le format de soumission

### Format de Soumission
- **PassengerId** : Identifiants des passagers de test
- **Transported** : Prédictions booléennes (True/False)
- **Fichier** : `submission.csv` prêt pour Kaggle

### Résultat Final
Le modèle Logistic Regression, entraîné sur l'ensemble des données avec un preprocessing soigneux, génère des prédictions avec une AUC de 0.7888 sur la validation, prometteuses pour la compétition.
